# Hartree-Fock Self-Consistent Field Theory for the Pauli-Fierz Hamiltonian (HF-PF)

"""Tutorial implementing a HF-PF program"""

__authors__ = "J. McTague, J. Foley, A. E. DePrince III"
__credits__ = "J. McTague, J. Foley, A. E. DePrince III"
__email__   = "foleyj10@wpunj.edu, deprince@fsu.edu"

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "11/04/2021"




# I. Theoretical Overview

The Pauli-Fierz Hamiltonian, $\hat{H}_{PF}$, is defined in the dipole approximation as:
$$\hat{H}_{PF} = \hat{H}_e + \hat{H}_p + \hat{H}_{ep} + \hat{H}_{dse}\tag{1}$$
***
### *Electronic Term*



The term $\hat{H}_e$ represents the electronic Hamiltonian and is given through:
$$\hat{H}_e = \sum^{N_e}_{i} \hat{T}_e(x_i) + \sum^{N_e}_i \sum^{N_N}_A \hat{V}_{eN}(x_i :X_A) + \sum^{N_e}_i \sum^{N_e}_j (\hat{V}_{ee}(x_i, x_j) + V_{N,N}\tag{2}$$
Where $\hat{T}_e(x_i)$ is the electronic kinetic operator for electron $i$, $\hat{V}_{eN}(x_i:X_A)$ is the attractive coulomb operator for electron $i$ and nucleus $A$, $\hat{V}_{ee}(x_i : x_J)$ is the repulsive coulomb operator for electrons $i$ and $j$, and $V_{N,N}$ is the total repulsive coulomb potential between all nuclei. When using the Born-Oppenheimer Approximation, the term $V_{N,N}$ is treated a constant. Additionally, we can neglect the nuclear kinetic energy, and the electron-nuclear attraction will depend upon the coordinates of the fixed nuclear coordinates.
***
### *Photonic Term*

The next term considered within the Pauli-Fierz Hamiltonian is the photonic Hamiltonian, $\hat{H}_p$, which is defined as:
$$\hat{H}_p = \omega\hat{b}^{\dagger{}}\hat{b}\tag{3}$$
Here, $\omega$ represents the frequency of a photon. 

***
### *Bilinear Coupling Term*
The third term in the Hamiltonian is a bilinear coupling term that describes coupling of the photonic and electronic degrees of freedom given by, $\hat{H}_{ep}$ and is defined as:
$$\hat{H}_{ep} = \sqrt{\frac{\omega}{2}}\left(\lambda \cdot (\hat{\mu} - \langle \mu \rangle )\right)\left(\hat{b}^{\dagger{}}+\hat{b}\right)\tag{6}$$

Where $\lambda$ describes the coupling strength, $\hat{\mu}$ is the dipole operator, and $\langle\mu\rangle$ is the molecular dipole expectation value in the ground state. Here, the expectation value has characteristic Cartesian components, denoted by $\xi$. A given dipole operator with $\xi$ components is defined through:
$$\hat{\mu}^{\xi} = \sum^{N_e}_i \hat{\mu}^\xi (x_i)+{\mu}_{nuc}^\xi \mid \xi \in \{x,y,z\}\tag{7}$$

Where $\mu^{\xi}(x_i)$ is defined as an operator that depends upon electronic coordinates, and $\mu_{nuc}^{\xi}$ is the nuclear dipole moment where the Cartesian components are treated as functions of the nuclear coordinate.

***
### *Dipole Self Energy Term*
The final component of the Hamiltonian, $\hat{H}_{dse}$ is the dipole self energy term, and is defined through:
$$\hat{H}_{dse} = \frac{1}{2}\left(\lambda \cdot (\hat{\mu} - \langle\mu\rangle)\right)^2\tag{8}$$

Here, $\lambda$ represents the polarization term, $\hat{\mu}$ represents the molecular dipole operator and $\langle\mu\rangle$ represents the ground state molecular expeectation value.
***
### *Hartree-Fock Reference*
Having discussed the terms that comprise our Hamiltonian, we can now begin our HF-PF procedure, which we initiate through the introduciton of a product wavefunction, $|R\rangle$. Here, we define $|R\rangle$ through the following: 
$$|R\rangle = |\Phi_{0}\rangle |0\rangle\tag{9}$$
Here, we have defined $|R\rangle$ as the product wavefunciton between an electronic Slater determinant, $\Phi_{0}$ and a zero-photon number state, $|0\rangle$.  We assume $|\Phi_{0}\rangle$ is a restricted reference. We can begin by performing a canonical RHF calculation to initialize the Slater determinant. 
With $|R\rangle$ defined, we can now apply Equation 9 to Equation 1, arriving at the following:
$$\langle R|\hat{H}_{ep}|R\rangle + \langle 0|\hat{H}_{p}|0\rangle + \langle \Phi_{0}| \hat{H}_{e} + \hat{H}_{dse}|\Phi_{0}\rangle\tag{10}$$

As we are using a zero-photon number state, we find that the $\hat{H}_{ep}$ and $\hat{H}_{p}$ terms both go to zero, leaving behind only the $\hat{H}_e$ and $\hat{H}_{dse}$. The resulting terms are seen in Equation 11.
$$\langle\Phi_{0}|\hat{H}_{e}|\Phi_{0}\rangle + \langle\Phi_{0}|\hat{H}_{dse}|\Phi_{0}\rangle\tag{11}$$

From this new equation, our HF-PF implementation differs from ordinary HF via the dipole self energy term--as $\hat{H}_{e}$ provides the ordinary RHF energy. 
***

***
### Dipole Self Energy Term Expansion
In order to better understand the role of the dipole self energy term, we will look at its expansion. 

$$\hat{H}_{dse} = \sum_{\xi, \xi^{'}}\sum_{i,j>i}\lambda^{\xi}\lambda^{\xi^{'}}\hat{\mu}^{\xi}(x_i)\hat{\mu}^{\xi^{'}}(x_j) \\- \frac{1}{2}\sum_{\xi,\xi^{'}}\sum_{i}\lambda^{\xi}\lambda^{\xi^{'}}\hat{Q}^{\xi\xi^{'}}(x_i) \\+ (\lambda\cdot\mu_{nuc}-\lambda\cdot\langle\mu\rangle)\sum_{\xi}\sum_{i}\lambda^{\xi}\hat{\mu}^{\xi'}(x_i) \\+ \frac{1}{2}(\lambda\cdot\mu_{nuc})^2-(\lambda\cdot\langle\mu\rangle)(\lambda\cdot\mu_{nuc}) + \frac{1}{2}(\lambda\cdot\langle\mu\rangle))^2\tag{12}$$

In this expansion of the dipolse self energy term, we observe several unique terms show up. We observe both 1- and 2-electron electronic dipole contributions, as well as 1-electron quadrupole contributions. The first two terms within this expansion effectively determine the presence of either the 2-electron dipole contribution or the 1-electron quadrupole contribution, given the respective values of the indices i and j. That is, we observe the 2-electron dipole contribution precisely when $i \neq j$, and the 1-electron quadrupole contribution when $i=j$. For clarification, we can rewrite $\hat{Q}^{\xi\xi^{'}}(x_i)$ as the following:
$$\hat{Q}^{\xi\xi^{'}}(x_i) = -\hat{\mu}^{\xi}(x_i)\hat{\mu}^{\xi^{'}}(x_i)$$
In addition to the index-dependent terms listed above, there is also a 1-electron electronic dipole contribution, scaled by the scalar equivalent to $\lambda\cdot\mu_{nuc}-\lambda\cdot\langle\mu\rangle$.
***

***
### Augmentation of HF Routine Using PF Terms
Having defined our Hamiltonian, we now discuss the modifications that the HF-PF approach makes to the canonical HF procedure. We begin by defining our Fock matrix as:
$$F_{\mu\nu} = H_{\mu\nu} + G_{\mu\nu}\tag{13}$$
$H_{\mu\nu}$ consists of the core Hamiltonian, $h_{\mu\nu}$, (which is equivalent to the potential energy plus the kinetic energy) with both of the 1-electron terms added onto it. $G_{\mu\nu}$ consists of two electron integrals multiplied by the density matrix, D, with our PF 2-electron terms added onto it. 
Therefore, we get:
$$H_{\mu\nu} = h_{\mu\nu}-\frac{1}{2}\sum_{\xi,\xi^{'}}\lambda^{\xi}\lambda^{\xi^{'}}Q_{\mu\nu}^{\xi\xi^{'}}+(\lambda\cdot\mu_{nuc}-\lambda\cdot\langle\mu\rangle)\sum_{\xi}\lambda^{\xi}\mu_{\mu\nu}^{\xi}\tag{14}$$
$$G_{\mu\nu}=(2(\mu\nu|\lambda\sigma)-(\mu\lambda|\nu\sigma))D_{\lambda\sigma}+\big(\sum_{\xi\xi^{'}}\lambda^{\xi}\lambda^{\xi^{'}}(2\mu_{\mu\nu}^{\xi}\mu_{\lambda\sigma}^{\xi^{'}}-\mu_{\mu\lambda}^{\xi}\mu_{\nu\sigma}^{\xi^{'}})\big)D_{\lambda\sigma}\tag{15}$$

Given these two formulas, we can construct a new formulation for the energy related to our HF modificaiton:
$$E_{HF-PF} = (F_{\mu\nu}+H_{\mu\nu})D_{\mu\nu} + V_{NN}+d_c\tag{16}$$
Where we take $d_c$ to be equal to:
$$d_c = \frac{1}{2}(\lambda\cdot\mu_{nuc})^2 - (\lambda\cdot\langle\mu\rangle)(\lambda\cdot\mu_{nuc})+\frac{1}{2}(\lambda\cdot\langle\mu\rangle)^2\tag{17}$$

Note the HF-PF energy contains two constant terms, $V_{N,N}$ and $d_{c}$, which represnt the total repulsive coulomb potential of all nuclei $(V_{N,N})$, as well as a constant term that arises from the consideration of our Dipole Self Energy Hamiltonian. 
***
***
### Outline of the Procedure
Now that we have addressed the theory related to our HF-PF approach, we will briefly outline the algorithm, which differs only in a few minor details from the canonical HF algorithm. This procedure will be demonstrated in a step-wise manner in the upcoming code segment--we are simply outlining the procedure here.
1. Compute kinetic, nuclear attraction, electron repulsion, dipole, and quadrupole integrals in AO basis
2. Perform canonical RHF calculation
3. Initialize ${\bf D}$ and $\langle \mu \rangle$ from canonical RHF wavefunction
4. Augment core Hamiltonian with the dipole and quadrupole terms in Eq. (14)
5. Augment the Fock matrix by contracting products of dipole integrals over current density matrix in Eq. (15)
6. Compute SCF energy through Eq. (16)
7. Diagonalize Fock matrix and update density matrix
8. Check for convergence; if not converged, return to step 5.




## II. Implementation

Using the above overview, let's write a RHF program using <span style="font-variant: small-caps"> Psi4 </span> and NumPy.  First, we need to import these Python modules: 

In [16]:
# ==> Import Psi4 & NumPy <==
import psi4
import numpy as np

*(This block appears in the basic RHF tutorial... do we need it?)*

Next, using what you learned in the previous tutorial module, set the following <span style="font-variant: small-caps"> Psi4 </span> and molecule options.

Memory & Output specifications:
- Give 500 Mb of memory to Psi4
- Set Psi4 output file to "output.dat"
- Set a variable `numpy_memory` to an acceptable amount of available memory for the working computer to use for storing tensors

Molecule definition:
*(Do we want to do formaldehyde instead?)*

- Define the "physicist's water molecule" (O-H bond length = 1.1 Angstroms, HOH bond angle = 104 degrees)
- Molecular symmetry C1

Computation options:
- basis set cc-pVDZ
- SCF type PK
- Energy convergence criterion to 0.00000001


In [17]:
# ==> Set Basic Psi4 Options <==
# Memory specification
psi4.set_memory(int(5e8))
numpy_memory = 2

# Set output file
psi4.core.set_output_file('output.dat', False)

# Define Physicist's water -- don't forget C1 symmetry!
mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

# Set computation options
psi4.set_options({'basis': 'cc-pvdz',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})

Since we will be writing our own, iterative RHF procedure, we will need to define options that we can use to tweak our convergence behavior.  For example, if something goes wrong and our SCF doesn't converge, we don't want to spiral into an infinite loop.  Instead, we can specify the maximum number of iterations allowed, and store this value in a variable called `maxiter`.  Here are some good default options for our program:
~~~python
MAXITER = 40
E_conv = 1.0e-6
~~~
These are by no means the only possible values for these options, and it's encouraged to try different values and see for yourself how different choices affect the performance of our program.  For now, let's use the above as our default.

In [18]:
# ==> Set default program options <==
# Maximum SCF iterations
MAXITER = 40
# Energy convergence criterion
E_conv = 1.0e-6

Before we can build our Fock matrix, we'll need to compute the following static one- and two-electron quantities:

- Electron repulsion integrals (ERIs) **I** between our AOs
- Overlap matrix **S**
- Orthogonalization matrix **A** = **S**$^{-1/2}$
- Kinetic energy integral matrix **T**
- Coulomb attraction integrals **V**
- Core Hamiltonian matrix **H = T + V**
- Dipole integrals **$\mu$** and **$\lambda$** vector dotted into them
- Quadrupole integrals **Q** and **$\lambda$** vector dotted into them
- Transformation vectors **C** from canonical HF calculation used to define guess Density Matrix **D**


In [19]:
# ==> Compute static 1e- and 2e- quantities with Psi4 <==
# ==> NEED TO ADD CALLS TO GET DIPOLE MATRIX AND QUADRUPOLE MATRIX! <==
# Class instantiation

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
mints = psi4.core.MintsHelper(wfn.basisset())

# Overlap matrix, S, and orthogonalization matrix, A
S = mints.ao_overlap()
A = mints.ao_overlap()
A.power(-0.5, 1.0e-16)
A = np.asarray(A)

# Number of basis Functions & doubly occupied orbitals
nbf = S.shape[0]
ndocc = wfn.nalpha()

print('Number of occupied orbitals: %3d' % (ndocc))
print('Number of basis functions: %3d' % (nbf))

# Memory check for ERI tensor
I_size = (nbf**4) * 8.e-9
print('\nSize of the ERI tensor will be {:4.2f} GB.'.format(I_size))
if I_size > numpy_memory:
    psi4.core.clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds allotted memory \
                     limit of %4.2f GB." % (I_size, numpy_memory))

# Build ERI Tensor
I = np.asarray(mints.ao_eri())

# Build core Hamiltonian
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H_0 = T + V


### Preparing a guess density matrix
C = np.asarray(wfn.Ca())
# use canonical RHF orbitals for guess CQED-RHF orbitals
Cocc = C[:, :ndocc]
# form guess density
D = np.einsum("pi,qi->pq", Cocc, Cocc)


### Preparing our lambda vector
lambda_vector = np.array([0.0, 0.0, 0.05])

### Preparing mu terms

### Nuclear Dipole terms for x,y,z
mu_nuc_x, mu_nuc_y, mu_nuc_z = mol.nuclear_dipole[0], mol.nuclear_dipole[1], mol.nuclear_dipole[2]

### Electronic Dipole terms for x,y,z in the atomic orbital basis
mu_ao_x, mu_ao_y, mu_ao_z = np.asarray(mints.ao_dipole()[0]), np.asarray(mints.ao_dipole()[1]), np.asarray(mints.ao_dipole()[2])

### Expectation values for the Electronic Dipole terms using our ordinary RHF valued density matrix
mu_exp_x, mu_exp_y, mu_exp_z = np.einsum("pq,pq->", 2 * mu_ao_x, D), np.einsum("pq,pq->", 2 * mu_ao_y, D), np.einsum("pq,pq->", 2 * mu_ao_z, D)
### Adding the nuclear dipole terms in
mu_exp_x += mu_nuc_x
mu_exp_y += mu_nuc_y
mu_exp_z += mu_nuc_z

### Creating a vector to represent the dipole moment
rhf_dipole_moment = np.array([mu_exp_x, mu_exp_y, mu_exp_z])

### Preparing a variable to represent our Electronic Dipole terms scaled by our polarization vector
l_dot_mu_el = (lambda_vector[0] * mu_ao_x) + (lambda_vector[1] * mu_ao_y) + (lambda_vector[2] * mu_ao_z)

### Doing the same, expect for the nuclear dipole moment
l_dot_mu_nuc = (lambda_vector[0] * mu_nuc_x) + (lambda_vector[1] * mu_nuc_y) + (lambda_vector[2] * mu_nuc_z)

### Once more, for the expectation value
l_dot_mu_exp = (lambda_vector[0] * mu_exp_x) + (lambda_vector[1] * mu_exp_y) + (lambda_vector[2] * mu_exp_z)

### Preparing our Quadrupole terms
Q_ao_xx = np.asarray(mints.ao_quadrupole()[0])
Q_ao_xy = np.asarray(mints.ao_quadrupole()[1])
Q_ao_xz = np.asarray(mints.ao_quadrupole()[2])
Q_ao_yy = np.asarray(mints.ao_quadrupole()[3])
Q_ao_yz = np.asarray(mints.ao_quadrupole()[4])
Q_ao_zz = np.asarray(mints.ao_quadrupole()[5])

### 1 electron quadrupole term
Q_PF = (-0.5 * lambda_vector[0] * lambda_vector[0] * Q_ao_xx) - (0.5 * lambda_vector[1] * lambda_vector[1] * Q_ao_yy) - (0.5 * lambda_vector[2] * lambda_vector[2] * Q_ao_zz)

### Necessary so that Q_ij == Q_ji holds true
Q_PF -= lambda_vector[0] * lambda_vector[1] * Q_ao_xy
Q_PF -= lambda_vector[0] * lambda_vector[2] * Q_ao_xz
Q_PF -= lambda_vector[1] * lambda_vector[2] * Q_ao_yz

d_PF = (l_dot_mu_nuc - l_dot_mu_exp) * l_dot_mu_el


### Preparing our dipole energy term
d_c = 0.5 * l_dot_mu_nuc ** 2 - l_dot_mu_nuc * l_dot_mu_exp + 0.5 * l_dot_mu_exp ** 2

### Now we want to add these terms to our core Hamiltonian, as previously defined
H = H_0 + Q_PF + d_PF


Number of occupied orbitals:   5
Number of basis functions:  24

Size of the ERI tensor will be 0.00 GB.


With all of the prerequisite variable having been established, we can continue onto our HF-PF procedure. We will begin by establishing our initial HF-PF energy using
the PF-Fock matrix and the guess density matrix obtained from the canonical HF calucation. We will also pull the convergence criteria for the energy and Density Matrix from the Psi4 options.

In [ ]:
print("\nStart SCF iterations:\n")
    t = time.time()
    E = 0.0
    Enuc = mol.nuclear_repulsion_energy()
    Eold = 0.0
    E_1el_crhf = np.einsum("pq,pq->", H_0 + H_0, D)
    E_1el = np.einsum("pq,pq->", H + H, D)
    print("Canonical RHF One-electron energy = %4.16f" % E_1el_crhf)
    print("CQED-RHF One-electron energy      = %4.16f" % E_1el)
    print("Nuclear repulsion energy          = %4.16f" % Enuc)
    print("Dipole energy                     = %4.16f" % d_c)

    # Set convergence criteria from psi4_options_dict
    if "e_convergence" in psi4_options_dict:
        E_conv = psi4_options_dict["e_convergence"]
    else:
        E_conv = 1.0e-7
    if "d_convergence" in psi4_options_dict:
        D_conv = psi4_options_dict["d_convergence"]
    else:
        D_conv = 1.0e-5



Now that we have our initial energies calculated and our convergences set, we can begin the SCF procedure. This procedure effectively follows the typical HF procedure, however, here we are considering two additional terms to append to the Fock matrix. These terms constitute the two electron contribution, seen in equation 15. These terms take on the shape of the J and K matrices that arise in the typical HF procedure. The created Fock matrix is comprised of all four of these terms. In addition, we will add on our core Hamiltonian--which has already been adapted to consider our CQED contributions--to the Fock matrix. We will use this Fock matrix as well as our initial Density matrix to calculate the self consistent field energy--in which we also consider our nuclear repulsion energy and dipole energy.

With our first energy calculated, we will now diagonalize our Fock matrix to gather a new coefficient matrix. We use this to build a new density matrix. 
With the new density matrix established, we must update our electronic dipole expectation values, to which we add the nuclear dipole terms as well. Additionally, we will update the expectation value of the polarization vector-electronic dipole dot product, as well as the 1 electron dipole term.

Finally, we will use these udpated terms to further augment our core Hamiltonian and our dipole energy. 



In [ ]:
t = time.time()
# maxiter
    maxiter = 500
    for SCF_ITER in range(1, maxiter + 1):

        # Build fock matrix: [Szabo:1996] Eqn. 3.154, pp. 141
        J = np.einsum("pqrs,rs->pq", I, D)
        K = np.einsum("prqs,rs->pq", I, D)

        # Pauli-Fierz 2-e dipole-dipole terms, line 2 of Eq. (12) in [McTague:2021:ChemRxiv]
        M = np.einsum("pq,rs,rs->pq", l_dot_mu_el, l_dot_mu_el, D)
        N = np.einsum("pr,qs,rs->pq", l_dot_mu_el, l_dot_mu_el, D)

        # Build fock matrix: [Szabo:1996] Eqn. 3.154, pp. 141
        # plus Pauli-Fierz terms Eq. (12) in [McTague:2021:ChemRxiv]
        F = H + J * 2 - K + 2 * M - N

        
        ### Check Convergence of the Density Matrix
        diis_e = np.einsum("ij,jk,kl->il", F, D, S) - np.einsum("ij,jk,kl->il", S, D, F)
        diis_e = A.dot(diis_e).dot(A)
        dRMS = np.mean(diis_e ** 2) ** 0.5

        # SCF energy and update: [Szabo:1996], Eqn. 3.184, pp. 150
        # Pauli-Fierz terms Eq. 13 of [McTague:2021:ChemRxiv]
        SCF_E = np.einsum("pq,pq->", F + H, D) + Enuc + d_c

        print(
            "SCF Iteration %3d: Energy = %4.16f   dE = % 1.5E   dRMS = %1.5E"
            % (SCF_ITER, SCF_E, (SCF_E - Eold), dRMS)
        )
        if (abs(SCF_E - Eold) < E_conv) and (dRMS < D_conv):
            break

        Eold = SCF_E

        # Diagonalize Fock matrix: [Szabo:1996] pp. 145
        Fp = A.dot(F).dot(A)  # Eqn. 3.177
        e, C2 = np.linalg.eigh(Fp)  # Solving Eqn. 1.178
        C = A.dot(C2)  # Back transform, Eqn. 3.174
        Cocc = C[:, :ndocc]
        D = np.einsum("pi,qi->pq", Cocc, Cocc)  # [Szabo:1996] Eqn. 3.145, pp. 139

        # update electronic dipole expectation value
        mu_exp_x = np.einsum("pq,pq->", 2 * mu_ao_x, D)
        mu_exp_y = np.einsum("pq,pq->", 2 * mu_ao_y, D)
        mu_exp_z = np.einsum("pq,pq->", 2 * mu_ao_z, D)

        mu_exp_x += mu_nuc_x
        mu_exp_y += mu_nuc_y
        mu_exp_z += mu_nuc_z

        # update \lambda \cdot <\mu>
        l_dot_mu_exp = (
            lambda_vector[0] * mu_exp_x
            + lambda_vector[1] * mu_exp_y
            + lambda_vector[2] * mu_exp_z
        )
        # Line 3 in full of Eq. (9) in [McTague:2021:ChemRxiv]
        d_PF = (l_dot_mu_nuc - l_dot_mu_exp) * l_dot_mu_el

        # update Core Hamiltonian
        H = H_0 + Q_PF + d_PF

        # update dipole energetic contribution, Eq. (14) in [McTague:2021:ChemRxiv]
        d_c = (
            0.5 * l_dot_mu_nuc ** 2
            - l_dot_mu_nuc * l_dot_mu_exp
            + 0.5 * l_dot_mu_exp ** 2
        )

        if SCF_ITER == maxiter:
            psi4.core.clean()
            raise Exception("Maximum number of SCF cycles exceeded.")
print("Total time for SCF iterations: %.3f seconds \n" % (time.time() - t))
print("QED-RHF   energy: %.8f hartree" % SCF_E)
print("Psi4  SCF energy: %.8f hartree" % psi4_rhf_energy)

Should convergence be met following any given cycle, the SCF procedure will truncate, providing the optimized values.
We will report the final terms back to the user in the form of a dictionary.

In [ ]:
rhf_one_e_cont = (
    2 * H_0
)  # note using H_0 which is just T + V, and does not include Q_PF and d_PF
rhf_two_e_cont = (
    J * 2 - K
)  # note using just J and K that would contribute to ordinary RHF 2-electron energy
pf_two_e_cont = 2 * M - N

SCF_E_One = np.einsum("pq,pq->", rhf_one_e_cont, D)
SCF_E_Two = np.einsum("pq,pq->", rhf_two_e_cont, D)
CQED_SCF_E_Two = np.einsum("pq,pq->", pf_two_e_cont, D)

CQED_SCF_E_D_PF = np.einsum("pq,pq->", 2 * d_PF, D)
CQED_SCF_E_Q_PF = np.einsum("pq,pq->", 2 * Q_PF, D)

assert np.isclose(
    SCF_E_One + SCF_E_Two + CQED_SCF_E_D_PF + CQED_SCF_E_Q_PF + CQED_SCF_E_Two,
    SCF_E - d_c - Enuc,
)

cqed_rhf_dict = {
    "RHF ENERGY": psi4_rhf_energy,
    "CQED-RHF ENERGY": SCF_E,
    "1E ENERGY": SCF_E_One,
    "2E ENERGY": SCF_E_Two,
    "1E DIPOLE ENERGY": CQED_SCF_E_D_PF,
    "1E QUADRUPOLE ENERGY": CQED_SCF_E_Q_PF,
    "2E DIPOLE ENERGY": CQED_SCF_E_Two,
    "CQED-RHF C": C,
    "CQED-RHF DENSITY MATRIX": D,
    "CQED-RHF EPS": e,
    "PSI4 WFN": wfn,
    "RHF DIPOLE MOMENT": rhf_dipole_moment,
    "CQED-RHF DIPOLE MOMENT": np.array([mu_exp_x, mu_exp_y, mu_exp_z]),
    "NUCLEAR DIPOLE MOMENT": np.array([mu_nuc_x, mu_nuc_y, mu_nuc_z]),
    "DIPOLE ENERGY": d_c,
    "NUCLEAR REPULSION ENERGY": Enuc,
}

return cqed_rhf_dict


In [20]:
# Compare to Psi4 - Need to think about how we will do this comparison!
SCF_E_psi = psi4.energy('SCF')
psi4.compare_values(SCF_E_psi, SCF_E, 6, 'SCF Energy')

NameError: name 'SCF_E' is not defined

### References
1. Pauli-Fierz Hamiltonian and CQED-RHF Equations
    - [[Haugland:2020:041043](https://journals.aps.org/prx/pdf/10.1103/PhysRevX.10.041043)] T. S. Haughland, E. Ronco, E. F. Kjonstad, A. Rubio, H. Koch, *Phys. Rev. X*, **10**, 041043 (2020)
    - [[DePrince:2021:094112]](https://aip.scitation.org/doi/10.1063/5.0038748) A. E. DePrince III, *J. Chem. Phys.* **154**, 094113 (2021).
2. Detailed CQED-RHF and CQED-CIS equations and overview of algorithm
    - [[McTague:2021:ChemRxiv](https://chemrxiv.org/engage/chemrxiv/article-details/611fa8d08a6faa13229c8be6)] J. McTague, J. J. Foley IV, *ChemRxiv*,
doi: 10.33774/chemrxiv-2021-0gpz8 (2021)